In [ ]:
#import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import factorial
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# # The files are labeled with respect to the measurement window and experiment length
# # In that order
# # 0.5ms_10s.txt measured the number of decays in 0.5ms windows for a total of 10 seconds
# # dt = 0.5ms
# # T = 10s
# data1 = pd.read_csv("0.5ms_10s.txt", names=["x"])
# data2 = pd.read_csv("0.5ms_100s.txt", names=["x"])
# data3 = pd.read_csv("0.5ms_1000s.txt", names=["x"])
# data4 = pd.read_csv("5ms_10s.txt", names=["x"])
# data5 = pd.read_csv("5ms_100s.txt", names=["x"])
# data6 = pd.read_csv("5ms_100s.txt", names=["x"])
# data7 = pd.read_csv("50ms_10000s.txt", names=["x"])
# data8 = pd.read_csv("500ms_50000s.txt", names=["x"])


filenames = ["0.5ms_10s.txt", "0.5ms_100s.txt", "0.5ms_1000s.txt",
             "5ms_10s.txt", "5ms_100s.txt", "5ms_100s.txt",
             "50ms_10000s.txt", "500ms_50000s.txt"]

dataframes = [pd.read_csv(filename, names=["x"]) for filename in filenames]


dt = [.5,.5,.5,5.,5.,5.,50.,500.]
T = [10.,100.,1000.,10.,100.,1000.,10000.,50000.]
# data1.head()

In [ ]:
# data1.describe()

In [ ]:
plt.hist(dataframes[0].x)

In [ ]:
# plt.hist(data7.x)

In [ ]:
# TODO:
# Fit the data with a poison function or gaussian if not possible 
# Extract mean (mu)
# show the plot

# define your own poisson distribution function
def poisson(k, a, lamb):
    return a*(lamb**k/factorial(k)) * np.exp(-lamb)
# lamb is the mean or expected value of the Poisson distribution

# now define your own normal distribution function
def gaus(x, a, mean, sigma):
    return a*np.exp(-0.5*((x-mean)/sigma)**2)

# Create a list of the data frames
dataframes = [data1, data2, data3, data4, data5, data6, data7, data8]
# lets make a empty list to store our mean values
mu = []
# Loop through each data frame and perform the plotting and curve fitting operations
# for i, df in enumerate(dataframes):
for i in range(len(dataframes)):
    df = dataframes[i]    
    # Get the data from the data frame
    data = df['x'].values
    
    # Define the histogram bins and calculate the bin heights
    bins = np.arange(min(data)-0.5, max(data)+1.5, 1.0)
    bin_heights, bin_edges = np.histogram(data, bins=bins, density=False)
    # Get the bin centers
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    if i < 7:
        # Fit the Poisson distribution to the data and make the plot with fit, keep the mean values in list mu[]
        # your code goes here .....
        # hint, to store mean values in mu[] use : mu.append(popt[1])


#     else:
#         # Try to fit with Poisson function
#         # If not fitted, try to fit the Gaussian function and keep the mean values in list mu[]
#         # your code goes here .....
#         # hint, to store mean values in mu[] use : mu.append(popt[1])
#     print("Mu, or average number of decays in",dt[i],"ms: ",mu[i]) 
    plt.show()


In [ ]:

def poisson(k, a, lamb):
    """poisson pdf, parameter lamb is the fit parameter"""
    return a*(lamb**k/factorial(k)) * np.exp(-lamb)
# lamb is the mean or expected value of the Poisson distribution
def gaus(x, a, mean, sigma):
    return a*np.exp(-0.5*((x-mean)/sigma)**2)

# Create a list of the data frames
# dataframes = [data1, data2, data3, data4, data5, data6, data7, data8]
mu = []
# Loop through each data frame and perform the plotting and curve fitting operations
# for i, df in enumerate(dataframes):
for i in range(len(dataframes)):
    df = dataframes[i]    
    # Get the data from the data frame
    data = df['x'].values

    # Define the histogram bins and calculate the bin heights
    print('min = ',min(data),'max = ', max(data))
    bins = np.arange(min(data)-0.5, max(data)+1.5, 1.0)
    bin_heights, bin_edges = np.histogram(data, bins=bins, density=False)
#     print(max(bin_heights))
#     print('bin_edges = ',bin_edges)
    # Get the bin centers
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
#     print('bin_centers = ',bin_centers)

    if i < 7:
        # Fit the Poisson distribution to the data
        popt, pcov = curve_fit(poisson, bin_centers, bin_heights, p0=[max(bin_heights),  np.mean(data)])
        print('Estimated normalizing constant a, lambda for DataFrame', i+1, ': a = ', popt[0],' mean (mu) =', popt[1])
        mu.append(popt[1])
        # Plot the data and the fitted curve
        plt.figure()
        plt.scatter(bin_centers, bin_heights, marker='o', label='data')
        xs = np.linspace(bin_centers[0], bin_centers[-1], 100)
        plt.plot(xs, poisson(xs, *popt), 'r-', lw=2, label='fit')
        plt.legend()
        plt.xlabel('x axis')
        plt.ylabel('y axis')
        plt.grid()
        plt.hist(data, bins=bins)

    else:
        # Fit the Gaussian distribution to the data
        popt, pcov = curve_fit(gaus, bin_centers, bin_heights, p0=[0.5, np.mean(data), np.std(data)], maxfev=5000)
        print('Estimated amp, mean, std for DataFrame', i+1, ':', popt[0],',', popt[1],',', popt[2])        
        mu.append(popt[1])
    
        # Plot the data and the fitted curve
        plt.figure()
        plt.scatter(bin_centers, bin_heights, marker='o', label='data')
        xs = np.linspace(bin_centers[0], bin_centers[-1], 100)
        plt.plot(xs, gaus(xs, *popt), 'r-', lw=2, label='smooth fit')
        plt.legend()
        plt.xlabel('x axis')
        plt.ylabel('y axis')
        plt.grid()
        plt.hist(data,  bins  = bins)

    print("Mu, or average number of decays in",dt[i],"ms: ",mu[i]) 
    plt.show()


In [ ]:
# TO DO 
# Plot mu versus Delta t from the different runs
# Fit with linear function and calculate the detected event rate

#Here we define a linear function to fit our mu vs dt plot 
def linear(x, a, b):
    return a*x + b

# Now lets plot the mean values using scatter plot
plt.scatter(......??)
#lets fit it to get the rate
popt, pcov = curve_fit(linear, dt, mu, p0=[1, 1])
print("Activity:",popt[0],"decays per millisecond")
xs = np.linspace(0, dt[-1], 100)
plt.plot(xs, linear(xs,*popt), c='r')
plt.title('??')
plt.xlabel("?")
plt.ylabel("?")
plt.grid()
plt.show()
